<a href="https://colab.research.google.com/github/cjudithrb/DataProcessing/blob/main/LaboSistemasRecomendaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Practica Calificada - Sistemas de Recomendaciones**

---



## **Pregunta 1:**
KNN utiliza una medida de similitud, como la distancia euclidiana, distancia coseno, entre otras, para identificar items o usuarios que son similares al usuario objetivo. Luego recomienda estos elementos basándose en las preferencias de estos "vecinos" cercanos, asumiendo que los gustos similares darán lugar a recomendaciones relevantes.

**Ventajas:**

- KNN es accesible y facil de implementar y entender, ya que solo requiere calcular la similitud entre usuarios o ítems.
- Es adaptable ya que se puede generar recomendaciones personalizadas basadas en gustos similares de otros usuarios o características de películas.

**Limitaciones:**

- En temas de escalabildiad, KNN se vuelve ineficiente y lento con grandes volúmenes de datos, ya que necesita calcular distancias con todos los vecinos.
- KNN puede verse afectado por datos ruidosos o irrelevantes, lo que puede perfudicar la calidad de las recomendaciones. En este caso para recomendar una pelicula que pueda ser del agrado del usuario.

## **Pregunta 2:**

In [1]:
# Descargar el dataset MovieLens 1M
!curl -o dataset.zip "https://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip dataset.zip
!ls -la


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5778k  100 5778k    0     0  7448k      0 --:--:-- --:--:-- --:--:-- 7446k
Archive:  dataset.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
total 5800
drwxr-xr-x 1 root root    4096 Oct 29 00:41 .
drwxr-xr-x 1 root root    4096 Oct 29 00:37 ..
drwxr-xr-x 4 root root    4096 Oct 25 13:20 .config
-rw-r--r-- 1 root root 5917549 Oct 29 00:41 dataset.zip
drwxr-x--- 2 root root    4096 Jan 29  2016 ml-1m
drwxr-xr-x 1 root root    4096 Oct 25 13:20 sample_data


In [20]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [4]:
# Cargar los datasets de calificaciones y películas
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python',
                      names=['userId', 'movieId', 'rating', 'timestamp'], encoding='latin-1')
movies  = pd.read_csv('ml-1m/movies.dat', sep='::',  header=None, engine='python',
                      names=['movieId', 'title', 'genres'], encoding='latin-1')

In [5]:
# Unir los datasets en base a 'movieId' para agregar los títulos
user_item_rating = pd.merge(ratings, movies[['movieId', 'title']], on='movieId')

# Seleccionar solo las columnas necesarias
user_item_rating = user_item_rating[['userId', 'title', 'rating']]

# Ordenar los datos por 'userId'
user_item_rating.sort_values(by='userId', inplace=True)

# Mostrar las primeras filas para verificar
print("#Ratings con titulos: ", user_item_rating.shape)
user_item_rating.head()

#Ratings con titulos:  (1000209, 3)


,userId,title,rating
0,1,One Flew Over the Cuckoo's Nest (1975),5
29,1,"Close Shave, A (1995)",3
30,1,Antz (1998),4
31,1,"Girl, Interrupted (1999)",4
32,1,Hercules (1997),4


In [ ]:
# Encontrar 2 usuarios con al menos 5 peliculas en comun
from collections import defaultdict

# Crear un diccionario para almacenar las películas calificadas por cada usuario
user_movies = defaultdict(set)
for _, row in user_item_rating.iterrows():
  user_movies[row['userId']].add(row['title'])

# Encontrar pares de usuarios con al menos 5 películas en común
pairs_users = []
for user1 in user_movies:
  for user2 in user_movies:
    if user1 < user2:  # Evitar duplicados (user1, user2) y (user2, user1)
      common_movies = user_movies[user1] & user_movies[user2]
      if len(common_movies) >= 5:
        pairs_users.append((user1, user2))

#print("Pares de usuarios con al menos 5 películas en común:")
#for user1, user2 in pairs_users:
#  print(f"Usuario {user1} y Usuario {user2}")

In [7]:
print("Pares de usuarios con al menos 5 películas en común:")
for user1, user2 in pairs_users[:2]:  # Mostrar solo los primeros 2 pares
  print(f"Usuario {user1} y Usuario {user2}")

Pares de usuarios con al menos 5 películas en común:
Usuario 1 y Usuario 2
Usuario 1 y Usuario 3


In [16]:
# LISTAR LAS CALIFICACIONES DE LOS 2 USUARIOS SELECCIONADOS

def get_common_ratings(user1, user2):
  """Obtiene las calificaciones en común de dos usuarios para las películas que ambos vieron."""

  user1_ratings = user_item_rating[user_item_rating['userId'] == user1]
  user2_ratings = user_item_rating[user_item_rating['userId'] == user2]

  common_movies = set(user1_ratings['title']).intersection(set(user2_ratings['title']))

  common_ratings = []
  for movie in common_movies:
    rating_user1 = user1_ratings[user1_ratings['title'] == movie]['rating'].iloc[0]
    rating_user2 = user2_ratings[user2_ratings['title'] == movie]['rating'].iloc[0]
    common_ratings.append({'MOVIES': movie, 'USER_'+str(user1): rating_user1, 'USER_'+str(user2): rating_user2})

  return common_ratings


# Ejemplo para el primer par de usuarios
if pairs_users:
  user1, user2 = pairs_users[0]
  common_ratings = get_common_ratings(user1, user2)

  # Mostrar en una tabla
  if common_ratings:
    df_common_ratings = pd.DataFrame(common_ratings)
    print(f"Calificaciones del Usuario {user1} y Usuario {user2}:\n")
    print(df_common_ratings)
  else:
    print(f"No hay películas en común entre el Usuario {user1} y el Usuario {user2}.")

Calificaciones del Usuario 1 y Usuario 2:

                                   MOVIES  USER_1  USER_2
0               Dead Poets Society (1989)       4       5
1            To Kill a Mockingbird (1962)       4       4
2                       Awakenings (1990)       5       4
3               Driving Miss Daisy (1989)       4       5
4              Saving Private Ryan (1998)       5       4
5  One Flew Over the Cuckoo's Nest (1975)       5       5
6                    Pleasantville (1998)       3       3


In [17]:
# CALCULAR LA SIMILITUD CON PEARSON
def pearson_correlation(user1_ratings, user2_ratings):
  """Calcula la correlación de Pearson entre las calificaciones de dos usuarios."""

  common_movies = set(user1_ratings.keys()).intersection(set(user2_ratings.keys()))
  if not common_movies:
    return 0  # No hay películas en común

  user1_common_ratings = [user1_ratings[movie] for movie in common_movies]
  user2_common_ratings = [user2_ratings[movie] for movie in common_movies]

  # Calcular la correlación de Pearson
  n = len(common_movies)
  sum_user1 = sum(user1_common_ratings)
  sum_user2 = sum(user2_common_ratings)
  sum_sq_user1 = sum([r**2 for r in user1_common_ratings])
  sum_sq_user2 = sum([r**2 for r in user2_common_ratings])
  sum_prod_user1_user2 = sum([r1 * r2 for r1, r2 in zip(user1_common_ratings, user2_common_ratings)])

  numerator = sum_prod_user1_user2 - (sum_user1 * sum_user2) / n
  denominator = (((sum_sq_user1 - (sum_user1**2) / n) * (sum_sq_user2 - (sum_user2**2) / n))**0.5)

  if denominator == 0:
    return 0

  correlation = numerator / denominator
  return correlation


if pairs_users:
  user1, user2 = pairs_users[0]

  # Obtener las calificaciones de los usuarios para las películas en común
  user1_ratings = {}
  user2_ratings = {}
  for _, row in user_item_rating.iterrows():
      if row['userId'] == user1:
          user1_ratings[row['title']] = row['rating']
      elif row['userId'] == user2:
          user2_ratings[row['title']] = row['rating']

  # Calcular la correlación de Pearson
  correlation = pearson_correlation(user1_ratings, user2_ratings)

  print(f"Correlación de Pearson entre el Usuario {user1} y el Usuario {user2}: {correlation}")

Correlación de Pearson entre el Usuario 1 y el Usuario 2: 0.4166666666666646


**Interpretacion:**

El valor de correlación de Pearson de 0.42 entre el Usuario 1 y el Usuario 2 indica una relación positiva moderada entre sus preferencias de películas. Esto sugiere que, en general, cuando al Usuario 1 le gusta una película, es probable que al Usuario 2 también le guste, y de la misma manea sel usuario 2 al usuario 1.

Sin embargo, la relación no es tan fuerte, digamos mas cercano a 1 para garantizar una fuerte correlacion en sus gustos, por lo que es muy probable que habran ciertas diferencias en sus preferencias.

## **Pregunta 3:**

In [18]:
# Agrupar las películas por título y contar el número de calificaciones
movie_ratings_count = user_item_rating.groupby('title')['rating'].count()

# Filtrar las películas con más de 50 calificaciones
movies_more_than_50_ratings = movie_ratings_count[movie_ratings_count > 50].index.tolist()

# Seleccionar una película aleatoriamente de la lista
import random

if movies_more_than_50_ratings:
  selected_movie = random.choice(movies_more_than_50_ratings)
  print(f"Película seleccionada con más de 50 valoraciones: {selected_movie}")
else:
  print("No se encontraron películas con más de 50 valoraciones.")

Película seleccionada con más de 50 valoraciones: You've Got Mail (1998)


In [ ]:
# Listar las peliculas de mayor a menor numero de calificaciones

# Agrupar las películas por título y contar el número de calificaciones
movie_ratings_count = user_item_rating.groupby('title')['rating'].count()

# Filtrar las películas con un máximo de 60 calificaciones
movies_less_than_60_ratings = movie_ratings_count[movie_ratings_count <= 40]

# Ordenar las películas por el número de calificaciones de mayor a menor
sorted_movies = movies_less_than_60_ratings.sort_values(ascending=False)

print("Películas con un máximo de 60 calificaciones (ordenadas de mayor a menor número de calificaciones):")
sorted_movies.head(40)

In [49]:
selected_movie = 'Mr. Wrong (1996)'

In [50]:
# Calcular la similitud de la pelicula seleccionada con otras dos usando similitud coseno

# Crear un diccionario de películas y sus calificaciones promedio
movie_avg_ratings = user_item_rating.groupby('title')['rating'].mean().to_dict()

# Crear una matriz de usuario-película con las calificaciones
user_movie_matrix = user_item_rating.pivot_table(index='userId', columns='title', values='rating')

# Reemplazar los valores NaN con 0
user_movie_matrix = user_movie_matrix.fillna(0)

# Obtener el vector de calificación de la película seleccionada
selected_movie_vector = user_movie_matrix[selected_movie]

In [51]:
# Seleccionar otras dos películas aleatorias
other_movie1 = 'Curdled (1996)'
other_movie2 = 'Beautiful (2000)'

In [52]:
# Seleccionar otras dos películas aleatoriamente de la lista de películas con más de 50 valoraciones
#other_movies = random.sample(movies_more_than_50_ratings, 2)

# Obtener los vectores de calificación de las otras dos películas
other_movie1_vector = user_movie_matrix[other_movie1]
other_movie2_vector = user_movie_matrix[other_movie2]

# Calcular la similitud de coseno entre la película seleccionada y las otras dos
similarity_movie1 = cosine_similarity([selected_movie_vector], [other_movie1_vector])[0][0]
similarity_movie2 = cosine_similarity([selected_movie_vector], [other_movie2_vector])[0][0]

print(f"Similitud de coseno entre '{selected_movie}' y '{other_movie1}': {similarity_movie1}")
print(f"Similitud de coseno entre '{selected_movie}' y '{other_movie2}': {similarity_movie2}")

Similitud de coseno entre 'Mr. Wrong (1996)' y 'Curdled (1996)': 0.06915141626285795
Similitud de coseno entre 'Mr. Wrong (1996)' y 'Beautiful (2000)': 0.0276177639577269


In [53]:
# Crear una lista para almacenar las calificaciones de la película seleccionada y las otras dos
movie_ratings_data = []

# Agregar las calificaciones de la película seleccionada
selected_movie_ratings = user_item_rating[user_item_rating['title'] == selected_movie][['userId', 'rating']].rename(columns={'rating': selected_movie})
movie_ratings_data.append(selected_movie_ratings)

# Agregar las calificaciones de la otra película 1
other_movie1_ratings = user_item_rating[user_item_rating['title'] == other_movie1][['userId', 'rating']].rename(columns={'rating': other_movie1})
movie_ratings_data.append(other_movie1_ratings)

# Agregar las calificaciones de la otra película 2
other_movie2_ratings = user_item_rating[user_item_rating['title'] == other_movie2][['userId', 'rating']].rename(columns={'rating': other_movie2})
movie_ratings_data.append(other_movie2_ratings)

# Fusionar las calificaciones en un solo DataFrame
merged_movie_ratings = movie_ratings_data[0]
for i in range(1, len(movie_ratings_data)):
    merged_movie_ratings = pd.merge(merged_movie_ratings, movie_ratings_data[i], on='userId', how='outer')

# Mostrar la tabla de calificaciones
print(f"Calificaciones de la película seleccionada ({selected_movie}) y otras dos:")
merged_movie_ratings.head(20)

Calificaciones de la película seleccionada (Mr. Wrong (1996)) y otras dos:


,userId,Mr. Wrong (1996),Curdled (1996),Beautiful (2000)
0,216,2.0,NaN,NaN
1,329,1.0,NaN,NaN
2,411,NaN,NaN,2.0
3,445,NaN,NaN,4.0
4,474,1.0,5.0,NaN
5,518,NaN,NaN,4.0
6,528,1.0,NaN,NaN
7,569,2.0,NaN,NaN
8,629,NaN,NaN,2.0
9,660,1.0,NaN,NaN


Los resultados indican que 'Mr. Wrong (1996)' es algo más similar a 'Curdled (1996)' (similitud de coseno de 0.069) que a 'Beautiful (2000)' (similitud de 0.028). Sin embargo, ambas similitudes son bajas, lo que sugiere que las tres películas tienen pocas características en común en el contexto del dataset, y, por tanto, no son muy parecidas en términos de las preferencias o atributos considerados.

## Pregunta 4

In [54]:
# Establecer el mínimo de calificaciones por película
min_ratings_per_movie = 5

# Filtrar las películas con al menos min_ratings_per_movie calificaciones
movie_counts = user_item_rating['title'].value_counts()
popular_movies = movie_counts[movie_counts >= min_ratings_per_movie].index

# Filtrar el dataset original
filtered_data = user_item_rating[user_item_rating['title'].isin(popular_movies)]
filtered_data.head()

,userId,title,rating
0,1,One Flew Over the Cuckoo's Nest (1975),5
29,1,"Close Shave, A (1995)",3
30,1,Antz (1998),4
31,1,"Girl, Interrupted (1999)",4
32,1,Hercules (1997),4


In [55]:
# DIVISION DE DATOS
def Random_Holdout(df, test_size=0.25):
    """Divide los datos asegurando que cada usuario tenga al menos una interacción en el conjunto de entrenamiento"""
    # Seleccionar al azar un porcentaje de interacciones por usuario para entrenamiento
    train_df = df.groupby('userId').apply(lambda x: x.sample(frac=1 - test_size, random_state=42)).reset_index(drop=True)
    # El resto va al conjunto de prueba
    test_df = pd.concat([df, train_df]).drop_duplicates(keep=False)

    return train_df, test_df

# Aplicar la función al dataset filtrado
train_df, test_df = Random_Holdout(filtered_data, test_size=0.3)

<ipython-input-55-b971037283ec>:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_df = df.groupby('userId').apply(lambda x: x.sample(frac=1 - test_size, random_state=42)).reset_index(drop=True)


In [ ]:
!pip install surprise

In [59]:
from surprise import Dataset, Reader, KNNBasic, accuracy

# Definir el rango de calificaciones
reader = Reader(rating_scale=(1, 5))

# Cargar datos de entrenamiento
trainset = Dataset.load_from_df(train_df[['userId', 'title', 'rating']], reader).build_full_trainset()

# Crear conjunto de prueba
testset = list(test_df[['userId', 'title', 'rating']].itertuples(index=False, name=None))

In [60]:
# Configurar opciones del modelo KNN
sim_options = {
    'name': 'msd',  # Mean Squared Difference (distancia euclidiana)
    'user_based': True  # Filtrado Usuario-Usuario
}

# Crear el modelo KNN
knn = KNNBasic(k=50, sim_options=sim_options)

# Entrenar el modelo
knn.fit(trainset)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
# Hacer predicciones sobre el conjunto de prueba
predictions = knn.test(testset)

# Convertir predicciones a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['userId', 'movieId', 'real_rating', 'predicted_rating', 'details'])

# Mostrar las predicciones principales para el usuario 324
user_324_predictions = predictions_df[predictions_df.userId == 324].sort_values(by='predicted_rating', ascending=False)
user_324_predictions.head(10)

### Evaluacion con Hit rate Global

In [ ]:
# Definir umbral de relevancia
relevant_threshold = 4.0

# Inicializar contadores
hits = 0
total_relevant = 0

# Iterar sobre todas las predicciones
for _, row in predictions_df.iterrows():
    real_rating = row['real_rating']
    predicted_rating = row['predicted_rating']

    # Considerar películas con calificación real ≥ umbral como relevantes
    if real_rating >= relevant_threshold:
        total_relevant += 1
        # Si la predicción también es ≥ umbral, es un acierto
        if predicted_rating >= relevant_threshold:
            hits += 1

# Calcular el Hit Rate Global
hit_rate_global = hits / total_relevant if total_relevant > 0 else 0
print(f"Hit Rate Global: {hit_rate_global:.2f}")



### Evaluacion con Hit rate por user

In [ ]:
# Inicializar lista para almacenar hit rates por usuario
hit_rates = []

# Agrupar predicciones por usuario
for user_id, group in predictions_df.groupby('userId'):
    hits = 0
    total_relevant = 0

    for _, row in group.iterrows():
        real_rating = row['real_rating']
        predicted_rating = row['predicted_rating']

        # Considerar películas relevantes
        if real_rating >= relevant_threshold:
            total_relevant += 1
            if predicted_rating >= relevant_threshold:
                hits += 1

    # Calcular Hit Rate para el usuario
    if total_relevant > 0:
        hit_rate_user = hits / total_relevant
        hit_rates.append(hit_rate_user)

# Calcular el Hit Rate promedio
hit_rate_avg_user = sum(hit_rates) / len(hit_rates) if len(hit_rates) > 0 else 0
print(f"Hit Rate promedio por usuario: {hit_rate_avg_user:.2f}")
